# PyOpenCL Parallel Patterns: Map/Elementwise

## Setup code

In [1]:
import pyopencl as cl
import pyopencl.array
import pyopencl.clrandom
import pyopencl.tools
import numpy as np

In [2]:
ctx = cl.create_some_context(interactive=True)
queue = cl.CommandQueue(ctx)

Choose platform:
[0] <pyopencl.Platform 'NVIDIA CUDA' at 0x1d5f790>
[1] <pyopencl.Platform 'Intel(R) OpenCL' at 0x1d5d818>
[2] <pyopencl.Platform 'Portable Computing Language' at 0x7f280ef385c0>
Choice [0]:
Choose device(s):
[0] <pyopencl.Device 'GeForce GTX TITAN X' on 'NVIDIA CUDA' at 0x1d6f680>
[1] <pyopencl.Device 'GeForce GTX TITAN X' on 'NVIDIA CUDA' at 0x1d70db0>
Choice, comma-separated [0]:
Set the environment variable PYOPENCL_CTX=':' to avoid being asked again.


In [3]:
n = 10**7
a = cl.clrandom.rand(queue, n, np.float32)
b = cl.clrandom.rand(queue, n, np.float32)

## A simple 'target application'

We would like to evaluate this linear combination:

In [4]:
c1 = 5*a + 6*b

A problem with this is that every single operator (all three of them--and easily more for complicated expressions) corresponds to a kernel call, which can lead to high overhead. Let's try and avoid that by stuffing the entire operation into one kernel, in turn saving lots of memory traffic:

In [5]:
from pyopencl.elementwise import ElementwiseKernel

Arguments: `ElementwiseKernel(ctx, c_args, operation)`

In [6]:
lin_comb = ElementwiseKernel(ctx,
        "float a, float *x, float b, float *y, float *c",
        "c[i] = a*x[i] + b*y[i]")

In [7]:
c2 = cl.array.empty_like(a)
lin_comb(5, a, 6, b, c2)

In [28]:
import numpy.linalg as la
print(la.norm(c1.get() - c2.get(), ord=np.inf))

9.536743e-07


## Timing ElementwiseKernel

Did this optimization pay off?

In [16]:
mem_pool = pyopencl.tools.MemoryPool(pyopencl.tools.ImmediateAllocator(queue))

a = cl.array.empty(queue, n, np.float32, allocator=mem_pool)
b = cl.array.empty(queue, n, np.float32, allocator=mem_pool)

cl.clrandom.fill_rand(a)
cl.clrandom.fill_rand(b)

In [19]:
from time import time

for i in range(3):
    c1 = 5*a + 6*b
    
queue.finish()
start_time = time()

for i in range(10):
    c1 = 5*a + 6*b
    
queue.finish()
print("elapsed: {0} s".format(time()-start_time))

elapsed: 0.013091564178466797 s


In [20]:
from time import time

for i in range(3):
    lin_comb(5, a, 6, b, c2)
    
queue.finish()
start_time = time()

for i in range(10):
    lin_comb(5, a, 6, b, c2)
    
queue.finish()
print("elapsed: {0} s".format(time()-start_time))

elapsed: 0.0056154727935791016 s


What happens without the memory pool?